In [ ]:
# It generates the dataset for mhui
# You need all data in the:
#     ./data/mhui/images
#     ./data/mhui/groundtruth
# It will crop the white parts and put the files in
#     ./data/mhui/cropped/images
#     ./data/mhui/cropped/groundtruth
# It will create the patches in:
#     ./data/mhui/generate/patches_rotation/images/
#     ./data/mhui/generate/patches_rotation/groundtruth/
# It will calculate the distances
#     ./data/mhui/generate/patches_rotation/distance/


In [1]:
import subprocess # subprocess.call(['./test.sh'])
import os
from os import listdir
import matplotlib.pyplot as plt
from scipy import ndimage, misc, stats
import numpy as np
from PIL import Image
from tqdm import tqdm
from os.path import isfile, join


In [2]:
THRESHOLD = 1
CURRENT_DIR = './data/mhui/'

In [3]:
# Cropp the white spaces

In [4]:
def crop_rows(original_image, original_groundtruth):
    width = original_image.shape[0]
    length = original_image.shape[1]
    for i in range(width):
        white_pixels_count = 0
        for j in range(length):
            if np.all(original_image[i][j] == 255):
                white_pixels_count = white_pixels_count + 1
        if white_pixels_count < THRESHOLD:
            break
    row_top = i

    for i in range(width):
        white_pixels_count = 0
        for j in range(length):
            if np.all(original_image[width - i - 1][j] == 255):
                white_pixels_count = white_pixels_count + 1
        if white_pixels_count < THRESHOLD:
            break
    row_bottom = width - i - 1        

    return original_image[row_top:row_bottom, :], original_groundtruth[row_top:row_bottom, :]

def crop_columns(original_image, original_groundtruth):
    width = original_image.shape[0]
    length = original_image.shape[1]

    for j in range(length):
        white_pixels_count = 0
        for i in range(width):
            if np.all(original_image[i][j] == 255):
                white_pixels_count = white_pixels_count + 1
        if white_pixels_count < THRESHOLD:
             break;        
    column_left = j

    for j in range(length):
        white_pixels_count = 0
        for i in range(width):
            if np.all(original_image[i][length - j -1] == 255):
                white_pixels_count = white_pixels_count + 1
        if white_pixels_count < THRESHOLD:
            break
    column_right = length - j - 1

    return original_image[:, column_left:column_right], original_groundtruth[:, column_left:column_right]

def crop_white_area(path, image_name):
    global index_cropping

    name = image_name.split(".")
    groundtruth_name = str(name[0]) + ".tif"
    img_org = Image.open(path + "/images/" + image_name)
    groundtruth_org = Image.open(path + "/groundtruth/" + groundtruth_name)
    groundtruth_org = groundtruth_org.convert('L')
    original_image = np.asarray(img_org)
    original_groundtruth = np.asarray(groundtruth_org)
    
    max_white_row_top = 0
    for j in range(original_image.shape[1]):
        if np.all(original_image[0][j] == 255):
            max_white_row_top = max_white_row_top + 1
    max_white_row_bottom = 0
    for j in range(original_image.shape[1]):
        if np.all(original_image[original_image.shape[0] - 1][j] == 255):
            max_white_row_bottom = max_white_row_bottom + 1 
    max_row = max(max_white_row_top, max_white_row_bottom)

    max_white_column_left = 0;
    for i in range(original_image.shape[0]):
        if np.all(original_image[i][0] == 255):
            max_white_column_left = max_white_column_left + 1
    max_white_column_right = 0;
    for i in range(original_image.shape[0]):
        if np.all(original_image[i][original_image.shape[1] - 1] == 255):
            max_white_column_right = max_white_column_right + 1
    max_column = max(max_white_column_left, max_white_column_right)


    if (max_row >= max_column):
        #print("Rows first!")
        cropped_image_rows, cropped_groundtruth_rows = crop_rows(original_image, original_groundtruth)
        cropped_image, cropped_groundtruth = crop_columns(cropped_image_rows, cropped_groundtruth_rows)
    else:
        #print("Columns first!")
        cropped_image_columns, cropped_groundtruth_columns = crop_columns(original_image, original_groundtruth)
        cropped_image, cropped_groundtruth = crop_rows(cropped_image_columns, cropped_groundtruth_columns)

    image = Image.fromarray(cropped_image, 'RGB')
    groundtruth = Image.fromarray(cropped_groundtruth, 'L')
    
    #save image
    image.save(path + "/cropped/images/" + image_name)
    groundtruth.save(path + "/cropped/groundtruth/" + groundtruth_name)
    index_cropping += 1
    #image.show()



In [ ]:
index_cropping = 0
print(CURRENT_DIR)

if not os.path.exists(CURRENT_DIR + "cropped"):
    os.makedirs(path + "cropped")
if not os.path.exists(CURRENT_DIR + "cropped/images"):
    os.makedirs(CURRENT_DIR + "cropped/images")
if not os.path.exists(CURRENT_DIR + "cropped/groundtruth"):
    os.makedirs(CURRENT_DIR + "cropped/groundtruth") 

original_path = CURRENT_DIR + "/images/"
for f in tqdm(listdir(original_path)):
    if f.endswith(".tiff"):
        crop_white_area(CURRENT_DIR, f)

  0%|          | 0/8 [00:00<?, ?it/s]

./data/mhui/


 62%|██████▎   | 5/8 [00:13<00:10,  3.42s/it]

In [ ]:
# Create patches

In [145]:
patch_size = 200
index = 0

def generate_images(groundtruth_dir, image_dir, image_name, output_dir, patch_size):
    global index
    
    name = image_name.split(".")
    groundtruth_name = str(name[0]) + ".tif"
    
    image_path = images_dir + image_name
    ground_t_path = ground_t_dir + groundtruth_name
    
    img_org = ndimage.imread(ground_t_path, mode='L')
    img_sat = ndimage.imread(image_path)

    for l, r in [(0,0), (0,200), (200,0), (200,200)]:
        
        patch_org = img_org[l:(l+200), r:(r+200)]
        patch_sat = img_sat[l:(l+200), r:(r+200)]
    
        org_name = '{}groundtruth/{}_{}.png'.format(output_dir,name[0], str(index).zfill(5))
        sat_name = '{}images/{}_{}.png'.format(output_dir,name[0], str(index).zfill(5))

        # '''
        rotation  = np.random.randint(0,4)
        patch_org = np.rot90(patch_org, rotation)
        patch_sat = np.rot90(patch_sat, rotation)
        # '''
        
        #print(org_name)
        if patch_org.shape == (200, 200):
            misc.imsave(org_name, patch_org)
            misc.imsave(sat_name, patch_sat)
        index += 1

In [146]:
index = 0
ground_t_dir = CURRENT_DIR + 'cropped/groundtruth/'
images_dir   = CURRENT_DIR + 'cropped/images/'
output_dir   = '{}/generate/patches_rotation/'.format(CURRENT_DIR)

if not os.path.exists(output_dir+'groundtruth'):
    os.makedirs(output_dir+'groundtruth')
if not os.path.exists(output_dir+'images'):
    os.makedirs(output_dir+'images')

for file in tqdm(listdir(images_dir)):
    
    generate_images(images_dir, ground_t_dir, file, output_dir, 200)

100%|██████████| 8/8 [00:00<00:00, 13.22it/s]


In [ ]:
# Calculate distances

In [150]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

def save_distances(input_path, output_path, img_name):
    global index_distance
    
    img_org = ndimage.imread(input_path+img_name, mode='L')
    img_org[img_org < 127] = 0
    img_org[img_org >= 127] = 255
    road_pxs     = np.argwhere(img_org != 0)
    non_road_pxs = np.argwhere(img_org == 0)
    output = np.zeros(img_org.shape, dtype=np.int)
    if len(road_pxs) == 0:
        return
    for coord in non_road_pxs:
        closest_px = road_pxs[closest_node(coord, road_pxs)]
        output[tuple(coord)] = (coord[0]-closest_px[0])**2 + (coord[1]-closest_px[1])**2
    
    misc.imsave(output_path+img_name, output)
    index_distance += 1
    

In [152]:
index_distance = 0
input_path   = '{}/generate/patches_rotation/groundtruth/'.format(CURRENT_DIR)
output_path   = '{}/generate/patches_rotation/distance/'.format(CURRENT_DIR)

if not os.path.exists(output_path):
    os.makedirs(output_path)
    
for f in tqdm(listdir(input_path)):
    save_distances(input_path, output_path, f)    